# This notebook demonstrates how to download the netcdf POES data files (in netcdf format) for a given date range (there are multiple files per day), process them to get auroral boundary (equatorward) and plot it!


In [1]:
import os
import datetime
from poes import dwnld_poes, get_aur_bnd

In [2]:
# dates to download raw poes files
sTimePOES = datetime.datetime( 2016,4,9 )
eTimePOES = datetime.datetime( 2016,4,9 )
# dir to store raw poes files
dayCount = (eTimePOES - sTimePOES).days + 1
# Loop through the days and download files
for inpDate in (sTimePOES + \
            datetime.timedelta(n) for n in range(dayCount)):
    poesDwnldObj = dwnld_poes.PoesDwnld(inpDate)
    # NOTE : set a proper outdir otherwise the data
    # is saved in the working directory by default
    poesFiles = poesDwnldObj.get_all_sat_data(outDir="/tmp/poes/raw")

downloading file from url-->http://satdat.ngdc.noaa.gov/sem/poes/data/processed/ngdc/uncorrected/full/2016/noaa18/poes_n18_20160409_proc.nc
moving file to destination folder poes_n18_20160409_proc.nc
outDir--> /tmp/poes/raw
downloading file from url-->http://satdat.ngdc.noaa.gov/sem/poes/data/processed/ngdc/uncorrected/full/2016/metop01/poes_m01_20160409_proc.nc
moving file to destination folder poes_m01_20160409_proc.nc
outDir--> /tmp/poes/raw
downloading file from url-->http://satdat.ngdc.noaa.gov/sem/poes/data/processed/ngdc/uncorrected/full/2016/noaa19/poes_n19_20160409_proc.nc
moving file to destination folder poes_n19_20160409_proc.nc
outDir--> /tmp/poes/raw
downloading file from url-->http://satdat.ngdc.noaa.gov/sem/poes/data/processed/ngdc/uncorrected/full/2016/metop02/poes_m02_20160409_proc.nc
moving file to destination folder poes_m02_20160409_proc.nc
outDir--> /tmp/poes/raw
downloading file from url-->http://satdat.ngdc.noaa.gov/sem/poes/data/processed/ngdc/uncorrected/full/

In [12]:
# Read data from the POES files
# and get the auroral boundary location
# by fitting a circle
poesRdObj = get_aur_bnd.PoesAur()
( poesAllEleDataDF, poesAllProDataDF ) = poesRdObj.read_poes_data_files(\
                                            poesRawDate=sTimePOES,\
                                            poesRawDir="/tmp/poes/raw/" )
# Or you can uncomment the line below and read the data!
# ( poesAllEleDataDF, poesAllProDataDF ) = poesRdObj.read_poes_data_files(poesFiles)
# Get for a given time get the closest satellite passes
# We can do this at multiple instances for a given time range/step
timeRange = [ poesAllEleDataDF["date"].min(),\
                 poesAllEleDataDF["date"].max() ]
# aurPassDF contains closest passes for a given time 
# for all the satellites in both the hemispheres!
aurPassDF = poesRdObj.get_closest_sat_passes( poesAllEleDataDF,\
                                    poesAllProDataDF, timeRange )
eqBndLocsDF = poesRdObj.get_nth_ele_eq_bnd_locs( aurPassDF, poesAllEleDataDF )

fileList not set! Working with poesRawDate & poesRawDir


In [13]:
eqBndLocsDF.head()

,sat,pole_mlat,pole_mlon,pole_mlt,equator_mlat,equator_mlon,equator_mlt,time
0,01,68.45,17.81,21.25,82.80,242.82,12.39,2016-04-09 00:30:00.653
1,15,68.35,290.06,15.51,52.85,154.55,6.75,2016-04-09 00:30:00.653
0,01,75.98,10.37,20.80,82.80,242.82,12.39,2016-04-09 01:00:00.653
1,15,68.35,290.06,15.51,52.85,154.55,6.75,2016-04-09 01:00:00.653
2,19,61.90,252.63,12.44,NaN,NaN,NaN,2016-04-09 01:00:00.653
